In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [2]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [3]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance31.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"), header=1, delim=",") |> DataFrame 
SDD = CSV.File(string(path,"SDD31.csv"),header=1,delim=",") |> DataFrame;

In [4]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)                          # 2021 global GD ($/yr)
es_ratio = global_sink/global_emi

emission_c = SDD[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = SDD[!, "sink ton/yr"]           # national CO2 sink (ton/yr)
Dsoc = SDD[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"];

In [5]:
cell_demand = 500000;

----

In [6]:
function EO_SS_Model(objfunc)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, MOI.Silent(), true)
    
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]
    
    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    
    # obj func calculation
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1)  # ncty*1 matrix
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1)  # ncty*1 matrix

    transS = Vector{AffExpr}(undef, ncty)
    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            arc_seq += arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        end
        transD[j] = arc_emi  # ncty*1 matrix
        transS[j] = arc_seq  # ncty*1 matrix
    end

    Allo_soc = proD ./ emission_c .* Dsoc
    
    
    SS = sum(Allo_soc - (proD + transD))
    EO = sum(proD - proS + transD)
    
    
    if objfunc == "Ecological"
        @objective(model, Min, EO);
    else
        @objective(model, Min, SS);
    end
      
    JuMP.optimize!(model)

    opt_x = JuMP.value.(x)
    opt_y = JuMP.value.(y)
    opt_obj = JuMP.objective_value(model);
    result = Dict(["x"=>opt_x, "obj"=>opt_obj, "y"=>opt_y])
    return result
end

EO_SS_Model (generic function with 1 method)

In [7]:
function overshoot_pct()
    model = Model(Gurobi.Optimizer)
    set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)
    set_optimizer_attribute(model, MOI.Silent(), true)
    
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, 0<=obj<=1)

    @variable(model, delta[1:ncty, 1:nproc], Bin)
    @variable(model, z[1:ncty, 1:nproc], Bin)
    @variable(model, r[1:ncty, 1:nproc], Bin)
    @variable(model, t[1:ncty, 1:nproc], Bin);
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink

    transD = Matrix{AffExpr}(undef, ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            emi = 0
            for j in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
            end
            transD[i,k] = emi
        end
    end
    
    slack = 1e-6
    M = 1e7
    
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty];
    
    EO = proD - proS + transD
    for i in 1:ncty
        for k in 1:nproc
            @constraint(model, EO[i,k] >= -M*r[i,k] + slack*t[i,k])
            @constraint(model, EO[i,k] <= -slack*r[i,k] + M*t[i,k])
            @constraint(model, z[i,k] + r[i,k] + t[i,k] == 1)
            @constraint(model, delta[i,k] => {x[i,k] >= 0.1})
        end
    end
    
    @constraint(model, obj*sum(delta) == sum(t))
    @objective(model, Min, obj)
    JuMP.optimize!(model)
    
    xx = JuMP.value.(x)
    yy = JuMP.value.(y)
    zz = JuMP.value.(z)
    rr = JuMP.value.(r)
    tt = JuMP.value.(t)
    solution = JuMP.objective_value(model)
    
    return Dict(["x"=>xx, "obj"=>solution, "y"=>yy, "z"=>zz, "t"=>tt, "r"=>rr])
    
end

overshoot_pct (generic function with 1 method)

---

In [8]:
function net_emi(x,y)
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    transS = Vector{Float64}(undef, ncty)
    transD = Vector{Float64}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            arc_seq += arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        end
        transD[j] = arc_emi  # ncty*1 matrix
        transS[j] = arc_seq  # ncty*1 matrix
    end
    
    EO = proD - proS + transD
    return(sum(EO))
end

net_emi (generic function with 1 method)

In [9]:
function soc_ss(x,y)
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1)  # ncty*1 matrix
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1)  # ncty*1 matrix

    
    transS = Vector{Float64}(undef, ncty)
    transD = Vector{Float64}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            arc_seq += arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        end
        transD[j] = arc_emi  # ncty*1 matrix
        transS[j] = arc_seq  # ncty*1 matrix
    end

    
    EO = sum(proD - proS + transD) 
    Allo_soc = proD ./ emission_c .* Dsoc
    SS = sum(Allo_soc - (proD + transD));
    
    return sum(SS)
end

soc_ss (generic function with 1 method)

In [10]:
function country_EO(x,y)
    
    proD = (x .* Matrix(regional_EF[:,2:end]))
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink
    
    
    transD = zeros(ncty, nproc)
    for k in 1:nproc
        for j in 1:ncty
            emi = 0
            for i in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans
            end
            transD[j,k] = emi
        end
    end
    
    
    EO = proD - proS + transD
    return proD + transD, proS
end

country_EO (generic function with 1 method)

In [11]:
function pct_count(x,y)
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink

    transD = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            emi = 0
            for j in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  
            end
            transD[i,k] = emi
        end
    end

    MEO = proD - proS + transD;
    
    indices = (MEO .>= 0.1) .|| (MEO .<= -0.1)
    ls = MEO[indices]
    total = length(ls)
    
    num_os = 0
    num_safe = 0
    for e in ls
        if e > 0
            num_os += 1
        elseif e < 0
            num_safe += 1
        end
    end
    
#     return proD, proS, transD

    return num_os,num_safe
end

pct_count (generic function with 1 method)

In [12]:
function cal_intm(EO_m)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)           
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    
    
    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]
    
    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    
    # obj func calculation
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1)  # ncty*1 matrix
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1)  # ncty*1 matrix

    
    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
        end
        transD[j] = arc_emi  # ncty*1 matrix
    end
    
    Allo_soc = proD ./ emission_c .* Dsoc
    SS = Allo_soc - (proD + transD)
    
    EO = sum(proD - proS + transD)
    @constraint(model, EO==EO_m)
    
    @objective(model, Min, sum(SS));

    JuMP.optimize!(model)
    opt_obj = JuMP.objective_value(model)
    opt_x = JuMP.value.(x)
    opt_y = JuMP.value.(y)
    
    return opt_obj, opt_x, opt_y
end
    

cal_intm (generic function with 1 method)

In [13]:
function ep_constraint(N, EO1, EO2)
    
    ep = (EO2 - EO1) / N
    lsss = []
    lseo = []
    lspct = []
    for i in 0:N
        EO_m = EO1 + i*ep
        obj_ss, opt_x, opt_y = cal_intm(EO_m)
        pct_m = pct_count(opt_x, opt_y)
        push!(lsss, obj_ss)
        push!(lseo, EO_m)
        push!(lspct, pct_m)
    end
    
    return lsss, lseo, lspct
end

ep_constraint (generic function with 1 method)

---

In [14]:
res_eo = EO_SS_Model("Ecological")
res_ss = EO_SS_Model("Social");

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26


In [43]:
res_eo

Dict{String, Any} with 3 entries:
  "obj" => 1.05664e6
  "x"   => [6200.0 0.0 … 0.0 0.0; 5579.65 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0;…
  "y"   => [0.0 0.0 … 0.0 6200.0; 0.0 0.0 … 0.0 5579.65; … ; 0.0 0.0 … 0.0 0.0;…

In [22]:
eo_x = DataFrame(res_eo["x"],:auto)
rename!(eo_x,process)
eo_x.index = countries
# CSV.write("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/EO_opt_X_0408.csv", eo_x)

"C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/EO_opt_X_0408.csv"

In [32]:
eo_y = res_eo["y"]
arc = []
for i in 1:ncty
    for j in 1:ncty
        for k in 1:nproc
            start = countries[i]
            stop  = countries[j]
            if eo_y[i,j,k] > 1 && i != j && [start, stop] ∉ arc
                push!(arc, [start, stop])
            end
        end
    end
end

column1 = [elem[1] for elem in arc]
column2 = [elem[2] for elem in arc]

df_loc = DataFrame(Column1 = column1, Column2 = column2)
rename!(df_loc,["from", "to"])
CSV.write("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/EO_loc.csv", df_loc)

"C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/EO_loc.csv"

In [35]:
ss_x = DataFrame(res_ss["x"],:auto)
rename!(ss_x,process)
ss_x.index = countries
# CSV.write("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/SS_opt_X_0408.csv", ss_x)

In [36]:
ss_y = res_ss["y"]
arc = []
for i in 1:ncty
    for j in 1:ncty
        for k in 1:nproc
            start = countries[i]
            stop  = countries[j]
            if ss_y[i,j,k] > 1 && i != j && [start, stop] ∉ arc
                push!(arc, [start, stop])
            end
        end
    end
end

column1 = [elem[1] for elem in arc]
column2 = [elem[2] for elem in arc]

df_loc = DataFrame(Column1 = column1, Column2 = column2)
rename!(df_loc,["from", "to"])
CSV.write("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/SS_loc.csv", df_loc)

"C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/SS_loc.csv"

In [38]:
OSPct = overshoot_pct()

Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 6 entries:
  "obj" => 0.210526
  "t"   => [-0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -0.0 0.0; … ; 0.0 0.0 … -0.0 -0.0; …
  "x"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.…
  "r"   => [0.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 1.0; … ; 1.0 1.0 … 0.0 1.0; 0.0 1.…
  "z"   => [1.0 0.0 … 0.0 0.0; 1.0 1.0 … 1.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 1.0 0.…
  "y"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.…

In [40]:
pct_x = DataFrame(OSPct["x"],:auto)
rename!(pct_x,process)
pct_x.index = countries
# CSV.write("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/PCT_opt_X_0408.csv", pct_x)

In [41]:
pct_y = OSPct["y"]
arc = []
for i in 1:ncty
    for j in 1:ncty
        for k in 1:nproc
            start = countries[i]
            stop  = countries[j]
            if pct_y[i,j,k] > 1 && i != j && [start, stop] ∉ arc
                push!(arc, [start, stop])
            end
        end
    end
end

column1 = [elem[1] for elem in arc]
column2 = [elem[2] for elem in arc]

df_loc = DataFrame(Column1 = column1, Column2 = column2)
rename!(df_loc,["from", "to"])
CSV.write("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/PCT_loc.csv", df_loc)

"C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/PCT_loc.csv"

----

In [20]:
EO1 = res_eo["obj"]
SS1 = res_ss["obj"]
ss_x = res_ss["x"]
ss_y = res_ss["y"]
eo_x = res_eo["x"]
eo_y = res_eo["y"];

In [21]:
EO2 = net_emi(ss_x, ss_y)

8.520654106727423e6

In [22]:
SS2 = soc_ss(eo_x, eo_y)

-1.0130200636800908e7

In [39]:
OSPct = overshoot_pct()

Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 6 entries:
  "obj" => 0.210526
  "t"   => [-0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -0.0 0.0; … ; 0.0 0.0 … -0.0 -0.0; …
  "x"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.…
  "r"   => [0.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 1.0; … ; 1.0 1.0 … 0.0 1.0; 0.0 1.…
  "z"   => [1.0 0.0 … 0.0 0.0; 1.0 1.0 … 1.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 1.0 0.…
  "y"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.…

In [40]:
pct_x = OSPct["x"]
pct_y = OSPct["y"];

In [41]:
num_os, num_safe = pct_count(pct_x,pct_y)
OS1 = num_os / (num_os+num_safe)

0.25

In [29]:
num_os, num_safe = pct_count(eo_x,eo_y)
OS_eo = num_os / (num_os+num_safe)

0.5

In [31]:
num_os, num_safe = pct_count(ss_x,ss_y)
OSss = num_os / (num_os+num_safe)

0.8484848484848485

In [42]:
EO_pct = net_emi(pct_x, pct_y)

3.667688221260466e6

In [44]:
SS_pct = soc_ss(pct_x, pct_y)

-9.51823344053983e6

In [63]:
N = 4

4

In [64]:
ep = (EO2 - EO1) / N
lsss = []
lseo = []
lspct = []
for i in 0:N
    EO_m = EO1 + i*ep
    obj_ss, opt_x, opt_y = cal_intm(EO_m)
    bad,good = pct_count(opt_x, opt_y)
    push!(lsss, obj_ss)
    push!(lseo, EO_m)
    push!(lspct, bad/(bad+good))
end

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26


In [65]:
lspct

5-element Vector{Any}:
 0.5
 0.5588235294117647
 0.6285714285714286
 0.6285714285714286
 0.8484848484848485

In [69]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/Tesla/";
#load social and ecological data and orgnize 
regional_EF = CSV.File(string(path,"EF_SC.csv"),header=1,delim=",", types=dtype) |> DataFrame 
capacity = CSV.File(string(path,"capacity.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance.csv"),header=1,delim=",") |> DataFrame 
SDD = CSV.File(string(path,"SDD.csv"),header=1,delim=",") |> DataFrame;

emission_c = SDD[!, "D"]         
sink_c = SDD[!, "S"]           
Dsoc = SDD[!, "Dsoc"]          

countries = capacity[!,"country"]
ncty = size(countries,1)                       
nproc = size(process,1);                         

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

emission_c = SDD[!, "D"]          # national CO2 emission (ton/yr)
sink_c = SDD[!, "S"]           # national CO2 sink (ton/yr)
Dsoc = SDD[!, "Dsoc"]           # national CO2 sink (ton/yr)

countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

tesla_eo = EO_SS_Model("Ecological")
tesla_ss = EO_SS_Model("Social")

EO1_tesla = tesla_eo["obj"]
SS1_tesla = tesla_ss["obj"]
ss_x_tesla = tesla_ss["x"]
ss_y_tesla = tesla_ss["y"]
eo_x_tesla = tesla_eo["x"]
eo_y_tesla = tesla_eo["y"]

EO2_tesla = net_emi(ss_x_tesla, ss_y_tesla)
SS2_tesla = soc_ss(eo_x_tesla, eo_y_tesla)

tesla_ss, tesla_eo, tesla_pct = ep_constraint(4, EO1_tesla, EO2_tesla)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26


(Any[-1.020627587733916e7, -1.1804541522433808e7, -1.2566932827329068e7, -1.300848653113937e7, -1.3256116817437839e7], Any[1.5783408217550563e6, 3.0503593332896587e6, 4.522377844824261e6, 5.994396356358863e6, 7.466414867893466e6], Any[(13, 13), (15, 13), (21, 11), (19, 11), (22, 5)])

In [70]:
perato2 = DataFrame([tesla_eo, tesla_ss, tesla_pct],:auto)
rename!(perato2,["EO", "SS", "PCT"])
CSV.write("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/PERATO_tesla_0406.csv", perato2)

"C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/result/PERATO_tesla_0406.csv"

In [ ]:
ep = (EO2_tesla - EO_tesla) / N
lsss0 = []
lseo0 = []
lspct0 = []
for i in 0:N
    EO_m = EO1 + i*ep
    obj_ss, opt_x, opt_y = cal_intm(EO_m)
    bad,good = pct_count(opt_x, opt_y)
    push!(lsss, obj_ss)
    push!(lseo, EO_m)
    push!(lspct, bad/(bad+good))
end